### Tic-Tac-Toe Library

#### Check the diagram provided for more clarity about design

This library defined in **lib** provides abstractions of Board, Player and Spot, essential for implementing Tic-Tac-Toe. Furthermore, it provides ability to extend it with new Player types, as well as new Game layer (for example, through UI), through abstractions of Game and output provider. To provide this configurable behavior, I capitalized on design patterns such as Factory, Abstract Factory and Template Method.

#### Basic abstractions

Board is a singleton—only one board is present and valid in one run of the game. Making it a singleton allows treating it is a registry that is always easy to access from any part of code. This is good because it is a fitting abstraction of the real-life tic-tac-toe game - board is always visible and accessible.

In [29]:
import org.jetbrains.kotlinx.tictactoe.Board

val board = Board.spots
Board.reset()
println(board[0][0].player)
println(board[0][0].isFree())

null
true


Board is accessible directly. It contains spots. Each spot has a player reference. When that reference is null, the spot is free, so prefer using isFree() instead of direct reference.

Board can be occupied. Below you see what happens within player makeMove method, because its direct invocation requires some form of input.

**It was easy to control that player names do not repeat, but this is deliberately skipped since player side uniquely identifies him.**

In [30]:
import org.jetbrains.kotlinx.tictactoe.Position
import org.jetbrains.kotlinx.tictactoe.player.StandardPlayer

val player_1 = StandardPlayer("X", "Player 1")
Board.occupy(player_1, Position(0, 0))

You can also attempt to occupy a non-existent position:

In [31]:
// SHOULD THROW EXCEPTION
Board.occupy(player_1, Position(10,10))

java.lang.ArrayIndexOutOfBoundsException: Index 10 out of bounds for length 3

Another player can try to occupy an occupied spot:

In [32]:
// SHOULD THROW EXCEPTION
val player_2 = StandardPlayer("O", "David")
Board.occupy(player_2, Position(0,0))

org.jetbrains.kotlinx.tictactoe.exceptions.SpotOccupiedException: This spot is occupied!

These exceptions are later caught and appropriately handled. See in tests.

#### Game status

There are many conditions for victory in tic-tac-toe:
1. All spots in row taken
2. All spots in column taken
3. All spots in diagonal taken

In an unlikely case where special levels are unlocked and game rules slightly modified, we might want to keep user interested by introducing different win conditions (i.e. you win if you get all the corners). This is easy to do - just extend from WinCondition.

In [33]:
import org.jetbrains.kotlinx.tictactoe.status.GameStatusCheck
import org.jetbrains.kotlinx.tictactoe.status.RowWinCondition

GameStatusCheck.addWinCondition(RowWinCondition())
Board.occupy(player_1, Position(0,1))
Board.occupy(player_1, Position(0,2))
GameStatusCheck.check()

WIN

There is a reason game status check takes no parameters—all checks and win conditions are completely decoupled from the game flow—they simply inspect the board state. This allows for easy reasoning about purpose and abilities of separate component. Game Status Check is not telling you who won, or in what turn, or anything similar; it simply determines if any win condition has been met.

We may also decide to modify end conditions to make the game more interesting—maybe reduce the number of steps in which the user may win, or eliminate some of the patterns. For this, we can extend from EndCondition.

In [34]:
import org.jetbrains.kotlinx.tictactoe.status.BoardFullEndCondition

GameStatusCheck.addEndCondition(BoardFullEndCondition())

In [35]:
import org.jetbrains.kotlinx.tictactoe.status.BoardFullEndCondition

Board.occupy(player_1, Position(1,0))
Board.occupy(player_1, Position(1,1))
Board.occupy(player_1, Position(1,2))
Board.occupy(player_1, Position(2,0))
Board.occupy(player_1, Position(2,1))
Board.occupy(player_1, Position(2,2))
BoardFullEndCondition().check()

true

#### How to use the library
##### to play the game, just run main

Examples above provide more insight into how the library works.

However, these are low-level, base abstractions that make the game work. The flow of the game is in the level above. You can see the Game class defined in _game_ package.
This class defines standard configurations for win conditions and end conditions but forces you to define how players will be configured, as well as where the game state must be output. You need to redefine the following elements to add a new access-layer (UI):
1. Define a new OutputProviderFactory. This will force you to define the full family of products: Board, Spot, Player and Message Output Provider. You will not use those directly, but the Game.start() method will.
2. If you want to define a new human player (like StandardPlayer), you define a new Player subclass.
3. Define a new Game subclass and define how the players will be configured (UserInterfaceGame)
4. When you define the new player type, add it to the Player factory as a key value pair. You are all set!

In [36]:
import game.cli.outputProvider.StandardOutputProviderFactory
import org.jetbrains.kotlinx.tictactoe.Spot
import org.jetbrains.kotlinx.tictactoe.outputProvider.BoardOutputProvider
import org.jetbrains.kotlinx.tictactoe.outputProvider.MessageOutputProvider
import org.jetbrains.kotlinx.tictactoe.outputProvider.OutputProviderFactory
import org.jetbrains.kotlinx.tictactoe.outputProvider.PlayerOutputProvider
import org.jetbrains.kotlinx.tictactoe.outputProvider.SpotOutputProvider
import org.jetbrains.kotlinx.tictactoe.player.Player
// THIS IS HOW YOU WOULD CREATE A NEW OUTPUT PROVIDER FAMILY
class SimpleMessageOutputProvider : MessageOutputProvider() {
    override fun output(message: String) {
        println(message)
    }

}

class SimpleSpotOutputProvider: SpotOutputProvider() {
    override fun output(spot: Spot) {
        StandardOutputProviderFactory.getSpotOutputProvider().output(spot)
    }
}

class SimpleBoardOutputProvider: BoardOutputProvider() {
    override fun output() {
        StandardOutputProviderFactory.getBoardOutputProvider().output()
    }

}

class SimplePlayerOutputProvider: PlayerOutputProvider() {
    override fun output(player: Player) {
        StandardOutputProviderFactory.getPlayerOutputProvider().output(player)
    }
}

object SimpleOutputProviderFactory: OutputProviderFactory() {
    override fun getBoardOutputProvider(): SimpleBoardOutputProvider {
        return SimpleBoardOutputProvider()
    }

    override fun getSpotOutputProvider(): SimpleSpotOutputProvider {
        return SimpleSpotOutputProvider()
    }

    override fun getPlayerOutputProvider(): SimplePlayerOutputProvider {
        return SimplePlayerOutputProvider()
    }

    override fun getMessageOutputProvider(): SimpleMessageOutputProvider {
        return SimpleMessageOutputProvider()
    }

}

For players, we will use non-interactive ones to avoid input.
First, enjoy two random players fight! No one knows who will win!

In [37]:
import game.Game
import org.jetbrains.kotlinx.tictactoe.player.RandomPlayer

class SimpleGame(): Game(SimpleOutputProviderFactory) {
    override fun configurePlayers(): List<Player> {
        return listOf(RandomPlayer("X", "Player 1"), RandomPlayer("O", "Player 2"))
    }
}
Board.reset()
SimpleGame().run()

Player Player 1 (X)'s turn!
---
   
---
   
---
   
---
Player Player 2 (O)'s turn!
---
   
---
   
---
X  
---
Player Player 1 (X)'s turn!
---
 O 
---
   
---
X  
---
Player Player 2 (O)'s turn!
---
 O 
---
X  
---
X  
---
Player Player 1 (X)'s turn!
---
 O 
---
X  
---
X O
---
Player Player 2 (O)'s turn!
---
 O 
---
X  
---
XXO
---
Player Player 1 (X)'s turn!
---
OO 
---
X  
---
XXO
---
Player Player 2 (O)'s turn!
---
OO 
---
X X
---
XXO
---
The winner is O: Player 2!
---
OOO
---
X X
---
XXO
---


Now we will see one random player against a smart player. This will be much less interesting since the smart player is killing it.

In [20]:
import org.jetbrains.kotlinx.tictactoe.player.SmartPlayer

class LessSimpleGame() : Game(SimpleOutputProviderFactory) {
    override fun configurePlayers(): List<Player> {
        return listOf(RandomPlayer("X", "Player 1"), SmartPlayer("O", "Smarter than David"))
    }
}
Board.reset()
LessSimpleGame().run()

Player Player 1 (X)'s turn!
---
   
---
   
---
   
---
Player Smarter than David (O)'s turn!
---
   
---
   
---
  X
---
Player Player 1 (X)'s turn!
---
   
---
 O 
---
  X
---
Player Smarter than David (O)'s turn!
---
 X 
---
 O 
---
  X
---
Player Player 1 (X)'s turn!
---
OX 
---
 O 
---
  X
---
Player Smarter than David (O)'s turn!
---
OXX
---
 O 
---
  X
---
Player Player 1 (X)'s turn!
---
OXX
---
 OO
---
  X
---
Player Smarter than David (O)'s turn!
---
OXX
---
 OO
---
 XX
---
The winner is O: Smarter than David!
---
OXX
---
OOO
---
 XX
---


And at last we can watch a very boring game that ends in draw, two smart players against each other:

In [23]:
import org.jetbrains.kotlinx.tictactoe.player.SmartPlayer

class BoringGame() : Game(SimpleOutputProviderFactory) {
    override fun configurePlayers(): List<Player> {
        return listOf(SmartPlayer("X", "Smarter than David too"), SmartPlayer("O", "Smarter than David"))
    }
}
Board.reset()
BoringGame().run()

Player Smarter than David too (X)'s turn!
---
   
---
   
---
   
---
Player Smarter than David (O)'s turn!
---
X  
---
   
---
   
---
Player Smarter than David too (X)'s turn!
---
X  
---
 O 
---
   
---
Player Smarter than David (O)'s turn!
---
XX 
---
 O 
---
   
---
Player Smarter than David too (X)'s turn!
---
XXO
---
 O 
---
   
---
Player Smarter than David (O)'s turn!
---
XXO
---
 O 
---
X  
---
Player Smarter than David too (X)'s turn!
---
XXO
---
OO 
---
X  
---
Player Smarter than David (O)'s turn!
---
XXO
---
OOX
---
X  
---
Player Smarter than David too (X)'s turn!
---
XXO
---
OOX
---
XO 
---
The game ended in a draw!
---
XXO
---
OOX
---
XOX
---
